# BE 1 traitement et protection de l'information

## 1. Huffman Algorithm

In [1]:
clear all; close all; clc;

1. We first write the function `DMS(A, P, m, n)` that computes a discrete source matrix of size $m \times n$ from an alphabet $A$ and a probability vector $P$.

In [2]:
%%file DMS.m
function DMS =  DMS(A, P, m, n)
    % generate discrete source matrix from alphabet A and probabilities P
    % m: number of rows, n: number of columns
    % DMS: discrete source matrix
    % returns DMS
    DMS = randsrc(m, n, [A; P]);
end

Created file 'C:\Users\Mathis\Desktop\ProtectInfo\DMS.m'.


We then try it with the following alphabet and probability vector, and compare the result with the one obtained with the `repmat` function.

In [3]:
A = [0, 1];
P = [0.5, 0.5];
m = 5;
n = 6;
X = DMS(A, P, m, n);

In [4]:
Y = repmat(A, m, n);

In [5]:
X
Y


X =

     1     0     0     0     1     1
     1     0     1     0     0     1
     0     1     1     1     1     0
     1     1     0     1     1     1
     1     1     1     1     1     0


Y =

     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1




2. That being done, we write the function `entropy(P)` that computes the entropy of a discrete random variable from its probability vector, and test it with a probability vector.

In [6]:
%%file entropy.m
function H = entropy(P)
    % compute entropy of discrete random variable
    % P: probability vector
    % H: entropy
    % returns H
    H = -sum(P .* log2(P));
end

Created file 'C:\Users\Mathis\Desktop\ProtectInfo\entropy.m'.


In [7]:
P = [.2, .5, .1, .2];
entropy(P)


ans =

    1.7610




3. We write the function `moybits(N, P)` that computes the average number of bits per symbol from a probability vector and a vector of codeword lengths.

In [8]:
%%file moybits.m
function moybits = moybits(N, P)
    % compute average number of bits per symbol
    % P: probability vector
    % N: length of codewords
    % moybits: average number of bits per symbol
    % returns moybits
    moybits = sum(P .* N);
end


Created file 'C:\Users\Mathis\Desktop\ProtectInfo\moybits.m'.


4. Now, we try Matlab functions `huffmanenco`

## 2 Source coding of words with memory or unknown probability of occurrence

1. Constructing the `LempelZivDic` function using `DMS` and Matlab functions.

In [96]:
%%file LempelZivDic.m

% function Dict = LempelZivDic(X, n)
%     %LempelZivDic - Generate a dictionary of length 2^n from the discrete source X.
%     %
%     % Syntax: Dict = LempelZivDic(X,n)
%     %
%     % The output of this function is a vector of structures (dict.mot and dict.code; a word of the code must be on n+1 bits).
%     % On parcourt tout le vecteur X pour chercher les mots uniques qui le constituent
%     % On les stocke dans un vecteur de structures
%     % On parcourt ensuite ce vecteur pour créer le code de chaque mot
%     % On stocke le code dans la structure correspondante
%     % On retourne le vecteur de structures
%     mots = [];
%     codes = [];
%     i = 1;
%     mot = dec2bin(X(i));

%     while i < length(X)

%         if ~ismember(mot, [Dict.mot])
%             mots(end + 1) = mot;
%             % Dict(end).code = dec2bin(i-1,n+1);
%         else
%             i = i + 1;
%             mot = strcat(mot, dec2bin(X(i)));
%         end

%     end

%     for mot = mots
%         codes(end + 1) = dec2bin(find([mots] == mot) - 1, n + 1);
%     end

%     Dict = struct('mot', mots, 'code', codes);
% end

function Dict = LempelZivDic(X, n)
    %LempelZivDic - Generate a dictionary of length 2^n from the discrete source X.
    %
    % Syntax: Dict = LempelZivDic(X,n)
    %
    % The output of this function is a vector of structures (dict.mot and dict.code; a word of the code must be on n+1 bits).
    % 
    % Input:
    % - X: vector representing the discrete source
    % - n: length of the code word (a word of the code must be on n+1 bits)
    % Output:
    % - Dict: vector of structures (dict.mot and dict.code)

    % Initialize the dictionary with the single symbols from the source
    Dict = struct('mot', num2cell(X), 'code', num2cell(1:length(X) - 1)');

    % Initialize the variables for the loop
    code_length = length(X);
    code_number = code_length;

    % Loop through the source to build the dictionary
    for i = 1:length(X)
        code_length = code_length + 1;
        code_word = X(i);
        j = i + 1;

        while j <= length(X) && code_length <= 2 ^ n
            code_word = [code_word, X(j)];
            j = j + 1;

            if ~ismember(code_word, [Dict.mot])
                Dict(code_number + 1) = struct('mot', {code_word}, 'code', code_number);
                code_number = code_number + 1;
                code_word = X(i);
                code_length = code_length + 1;
            end

        end

    end

    % Remove any extra entries in the dictionary if necessary
    if code_length > 2 ^ n
        Dict = Dict(1:2 ^ n);
    end

end


Created file 'C:\Users\Mathis\Desktop\ProtectInfo\LempelZivDic.m'.


In [98]:
n1 = 4
X = (rand(1, 50) > 0.1);

Dict1 = LempelZivDic(X, n1);
Dict1(1).mot



  File C:\Users\Mathis\Desktop\ProtectInfo\LempelZivDic.m, line 51, in LempelZivDic
Array dimensions of input '4' must match those of input '2', or be scalar.



2. Constructing the `LempelZivEnCo` function using `LempelZivDic` and Matlab functions.

In [10]:
%%file LempelZivEnCo.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% function X = LempelZivEnCo(X,Dict)
% LempelZivEnCo - Encodes a DMS X using the dictionary Dict.
% returns the encoded DMS X.
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function E = LempelZivEnCo(X,Dict)
    % initialize variables
    E = [];
    n = log2(length(Dict));
    X = X(:)';
    X = [X, zeros(1, n)];
    i = 1;

    % loop over the DMS
    while i <= length(X) - n
        % find the longest word in the dictionary that matches the current
        % word
        for j = 1:length(Dict)
            if X(i:i+n) == Dict(j).mot
                E = [E, Dict(j).code];
                i = i + n + 1;
                break
            end
        end
    end

    % remove the last n bits
    E = E(1:end-n);
end

Created file 'C:\Users\Mathis\Desktop\ProtectInfo\LempelZivEnCo.m'.


3. Constructing the `LempelZivDeCo` function using `LempelZivDic` and Matlab functions.

In [14]:
%%file LempelZivDeco.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% function Y = LempelZivDeco(X,Dict)
% LempelZivDeco - Decodes a received message Y using the Dict dictionary.
% returns the decoded DMS Y.
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function Y = LempelZivDeco(X,Dict)
    % initialize variables
    Y = [];
    n = log2(length(Dict));
    X = X(:)';
    X = [X, zeros(1, n)];
    i = 1;

    % loop over the DMS
    while i <= length(X) - n
        % find the longest word in the dictionary that matches the current
        % word
        for j = 1:length(Dict)
            if X(i:i+n) == Dict(j).code
                Y = [Y, Dict(j).mot];
                i = i + n + 1;
                break
            end
        end
    end

    % remove the last n bits
    Y = Y(1:end-n);
end

Created file 'C:\Users\Mathis\Desktop\ProtectInfo\LempelZivDeco.m'.


3. Applying functions on a large sequence (> 1000 bits)

5. Given program to test the functions. Need to add the results in comments.

In [84]:
%Programme I
n1 = 4;
n2 = 11;
n3 = 6;
rand('state', 0);
X = (rand(1, 50000) > 0.1);

Dict1 = LempelZivDic(X, n1);
longmot1 = Dict1(2 ^ n1).mot
lgmot1 = length(Dict1(2 ^ n1).mot)

Dict2 = LempelZivDic(X, n2);
lgmot2 = length(Dict2(2 ^ n2).mot)

X1 = LempelZivEnCo(X, Dict1);
R1 = length(X1) / length(X)

X2 = LempelZivEnCo(X, Dict2);
R2 = length(X2) / length(X)

Y = (rand(1, 1000) > 0.5);
Y1 = LempelZivEnCo(Y, Dict2);
RY1 = length(Y1) / length(Y)

DictY = LempelZivDic(Y, 6);
Y2 = LempelZivEnCo(Y, DictY);
RY2 = length(Y2) / length(Y)

X = X(1:1000);

X2 = LempelZivEnCo(X, Dict2);
R2 = length(X2) / length(X)

X3 = LempelZivDeco(X2, Dict2);

% The output of LempelZivDeco is a "string"
for i = 1:length(X3)
    X4(i) = str2num(X3(i))
end

BER = sum((X(1:length(X3)) - X4) .^ 2) %BER


  File C:\Users\Mathis\Desktop\ProtectInfo\LempelZivDic.m, line 20, in LempelZivDic
Unrecognized function or variable 'Dict'.

